In [184]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


In [185]:
ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()
TheQueues = []
TheCTStats = []
TheDTStats = []
TheResources = []


# Classes

In [186]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}
        self.bike_list = []
        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available
            self.bike_list.append(bike_id)
            
    def rent_bike(self):
        # Request a bike from the station
        if self.level > 0:
            if self.bike_list:
                random_index = np.random.randint(0, len(self.bike_list))
                bike_id = self.bike_list.pop(random_index)
                self.level -= 1
                return bike_id
        return None
    
    def return_bike(self, bike_id):
        # # Return a bike to the station
        self.bike_list.append(bike_id)
        self.level += 1

    def Get_Bike_List(self):
        return self.bike_list
    
class Customer:
    def __init__(self, customer_id, start_station, bike = None):
        self.customer_id = customer_id
        self.start_station = start_station
        self.end_id = 0
        self.station_level = 0
        self.bike = bike

    def rent_bike(self):
        station = StationDict[self.start_station]
        self.station_level = station.level - 1
        self.bike = station.rent_bike()
        print("(CUSTOMER ARRIVAL) Customer ID: {} | Bike ID {} || Station ID: {} -> Level: {}".format(self.customer_id, 
                                                                                                   self.bike, 
                                                                                                   self.start_station, 
                                                                                                   station.level))
        SimFunctions.Schedule(Calendar, "Departure", 2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))
        
    def return_bike(self, end_station, bike):
        destination_station = StationDict[end_station]
        self.station_level = destination_station.level
        destination_station.return_bike(bike)
        

# Functions

In [191]:
def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 5)
    if U < (1/3):
        station_id = 1
    elif U < (2/3) and U > (1/3):
        station_id = 2
    else:
        station_id = 3
    return station_id

def Customer_Arrival(station_id, empty_error, CustomerList):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]
    customer_id = NextCustomerID()
    customer = Customer(customer_id, station_id)
    CustomerList.append(customer)
    customer.start_station = station_id

# Station is empty
    if station.level <= 0:
        print(f"    EMPTY -- Customer {customer.customer_id} Arrives at STATION {station.id} -- EMPTY")
        #SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        empty_error += 1
    else:
        customer.rent_bike()

    return Customer_Arrival_Rate(station_id), empty_error, customer

def Bike_Arrival(end_id, Full_Error, CustomerList):
    last_customer = CustomerList[-1]
    last_customer.end_id = end_id
    end_station = StationDict[end_id] 

    for customer in CustomerList:
        if customer.end_id == end_id and customer.bike is not None:
            customer.return_bike(end_id, customer.bike)
            print(" (BIKE RETURNED) Customer ID: {} | Bike ID: {} || From: {} | To: {} -> Level {}".format(customer.customer_id,
                                                                                                                     customer.bike,
                                                                                                                     customer.start_station,
                                                                                                                     end_station.id,
                                                                                                                     end_station.level))
            CustomerList.remove(customer)
            return Full_Error

    if end_station.level >= end_station.capacity:
#Station is Full
        print(f"    FULL -- STATION {end_station.id} -- FULL")
        Full_Error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(2, 5))
        return Full_Error
    
    return Full_Error



def Retrial(end_id, Full_Error, CustomerList):
    print("Retrial")
        
    last_customer = CustomerList[-1]
    last_customer.end_id = end_id
    end_station = StationDict[end_id]

    if end_station.level < end_station.capacity:
        for customer in CustomerList:
            if customer.end_id == end_id and customer.bike is not None:
                print("     Bike has succesfully retried and returned bike to Station", end_station.id)
                customer.return_bike(end_id, customer.bike)
                print(" (RETRIAL: BIKE RETURNED) Customer ID: {} | Bike ID: {} || From: {} | To: {} -> Level {}".format(customer.customer_id,
                                                                                                                    customer.bike,
                                                                                                                    customer.start_station,
                                                                                                                    end_station.id,
                                                                                                                    end_station.level))
                print("-------------------------------------------------------------------------")
                CustomerList.remove(customer)
                return Full_Error
    else:
        # Station full or bike not found, schedule a retrial event
        print(f"    FULL -- STATION {end_station.id} -- FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(2, 5))
        Full_Error += 1
        return Full_Error
    return Full_Error


def Departure(station_id, customer):
    station = StationDict[station_id]
    end_id = Destination(customer)
    customer.end_id = end_id
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 15 *SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))
    return end_id


def Destination(customer):
    
    U = SimRNG.Uniform(0, 1, 3)
    if U < (1/3):
        end_id = 1
    elif U < (2/3) and U > (1/3):
        end_id = 2
    else:
        end_id = 3

    end_station = StationDict[end_id]
    print("(DEPARTING) Customer ID: {} | Bike ID: {} || From: {} -> Level {} | To: {} -> Level {}".format(customer.customer_id,
                                                                                            customer.bike,
                                                                                            customer.start_station, 
                                                                                            customer.station_level, 
                                                                                            end_station.id, 
                                                                                            end_station.level))
    print("-------------------------------------------------------------------------")
    return end_id


# Simulation

In [192]:
def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter


ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

CustomerList = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1
Full_Error = 0
Empty_Error = 0

for reps in range(0, 1, 1):
    
    inital_station_id = np.random.random_integers(1, 3)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=5, capacity=5), Station(station_id=2, level=35, capacity=40), Station(station_id=3, level=25, capacity=25)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, CustomerList)


    for hours in [8, 8.3, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            
            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, customer = Customer_Arrival(station_id, Empty_Error, CustomerList)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id, customer)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, CustomerList)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, CustomerList)
               
    for station_id, station in StationDict.items():
        print(f"Station {station_id} Bike List: {station.Get_Bike_List()}")

    print(f"Num of Full Errors: {Full_Error}")
    print(f"Num of Empty Errors: {Empty_Error}")


(CUSTOMER ARRIVAL) Customer ID: 1 | Bike ID 2-8 || Station ID: 2 -> Level: 34
Clock Hour | 8
Minute | 1
(DEPARTING) Customer ID: 92 | Bike ID: 2-23 || From: 2 -> Level 28 | To: 3 -> Level 25
-------------------------------------------------------------------------
Minute | 2
(CUSTOMER ARRIVAL) Customer ID: 2 | Bike ID 2-30 || Station ID: 2 -> Level: 33
Minute | 3
(DEPARTING) Customer ID: 2 | Bike ID: 2-30 || From: 2 -> Level 33 | To: 3 -> Level 25
-------------------------------------------------------------------------
Minute | 4
(CUSTOMER ARRIVAL) Customer ID: 3 | Bike ID 2-12 || Station ID: 2 -> Level: 32
Minute | 5
(DEPARTING) Customer ID: 3 | Bike ID: 2-12 || From: 2 -> Level 32 | To: 3 -> Level 25
-------------------------------------------------------------------------
Minute | 6
 (BIKE RETURNED) Customer ID: 2 | Bike ID: 2-30 || From: 2 | To: 3 -> Level 26
Minute | 7
(CUSTOMER ARRIVAL) Customer ID: 4 | Bike ID 2-34 || Station ID: 2 -> Level: 31
Minute | 8
(DEPARTING) Customer I

In [193]:
for station_id, station in StationDict.items():
    print(f"Station {station_id} Bike List: {station.Get_Bike_List()}")

print(f"Num of Full Errors: {Full_Error}")
print(f"Num of Empty Errors: {Empty_Error}")


Station 1 Bike List: ['2-30']
Station 2 Bike List: ['2-3', '2-4', '2-11', '2-14', '2-16', '2-25', '2-27', '2-28', '2-29', '2-31', '2-35', '2-17', '1-5', '2-8', '3-2', '3-1', '2-9', '3-23', '3-24', '2-18', '3-17', '2-6', '2-22', '2-33']
Station 3 Bike List: ['3-4', '3-6', '3-7', '3-8', '3-10', '3-11', '3-16', '3-18', '3-19', '3-21', '2-12', '1-4', '2-1', '3-25', '2-19', '2-26', '1-1', '3-20', '3-14', '3-5', '2-20', '2-34', '2-13', '2-2', '3-12', '1-3', '2-32', '1-2', '3-15']
Num of Full Errors: 1
Num of Empty Errors: 15
